In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
train_df = pd.read_csv("data/train_new.csv")

# Drop the null rows
train_df = train_df.dropna()
train_df = train_df.drop(columns = ['Date of Joining'])

train_df.count()

Gender                  18590
Company Type            18590
WFH Setup Available     18590
Designation             18590
Resource Allocation     18590
Mental Fatigue Score    18590
Burn Rate               18590
dtype: int64

In [3]:
train_df

,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,1,0,0,2.0,3.0,3.8,0.16
1,0,0,1,1.0,2.0,5.0,0.36
3,0,0,1,1.0,1.0,2.6,0.20
4,1,0,0,3.0,7.0,6.9,0.52
5,0,1,1,2.0,4.0,3.6,0.29
...,...,...,...,...,...,...,...
22743,1,1,1,1.0,3.0,6.0,0.48
22744,0,1,0,3.0,7.0,6.2,0.54
22746,1,1,1,3.0,6.0,6.7,0.59
22748,1,0,0,2.0,5.0,5.9,0.52


In [4]:
selected_features = train_df.drop(columns = ['Burn Rate'])

selected_features

,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score
0,1,0,0,2.0,3.0,3.8
1,0,0,1,1.0,2.0,5.0
3,0,0,1,1.0,1.0,2.6
4,1,0,0,3.0,7.0,6.9
5,0,1,1,2.0,4.0,3.6
...,...,...,...,...,...,...
22743,1,1,1,1.0,3.0,6.0
22744,0,1,0,3.0,7.0,6.2
22746,1,1,1,3.0,6.0,6.7
22748,1,0,0,2.0,5.0,5.9


In [5]:
#split train test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, train_df['Burn Rate'], 
                                                    random_state=1, stratify = train_df['Burn Rate'])

In [8]:
#Scale data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_scale_train = X_scaler.transform(X_train)
X_scale_test = X_scaler.transform(X_test)


In [9]:
#Train the model 
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn import utils 

lin_regr_model = LinearRegression()
lin_regr_model

LinearRegression()

In [10]:
#fit model

lin_regr_model.fit(X_train, y_train)

LinearRegression()

In [11]:
#Print Scores 
print(f"Training Data Score: {lin_regr_model.score(X_train, y_train)}")
print(f"Testing Data Score: {lin_regr_model.score(X_test, y_test)}")

Training Data Score: 0.9212250346164508
Testing Data Score: 0.9189457447498501


In [12]:
predictions = lin_regr_model.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [0.26291879 0.25952856 0.56807454 0.35155855 0.3564176  0.00536597
 0.64550972 0.31858356 0.42900025 0.26280427]
First 10 Actual labels: [0.28, 0.27, 0.54, 0.43, 0.39, 0.03, 0.64, 0.36, 0.35, 0.27]


In [14]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True).sample(10)

,Prediction,Actual
3558,0.824248,0.80
1118,0.240591,0.18
4515,0.254850,0.21
3220,0.322049,0.35
1068,0.549259,0.55
1357,0.605802,0.69
3050,0.270908,0.26
833,0.466622,0.52
4304,0.606435,0.67
2738,0.510358,0.47


In [19]:
#SVR 

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([('scaler', StandardScaler()), ('svr', SVR())])
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set
pipe.fit(X_train, y_train)
Pipeline(steps=[('scaler', StandardScaler()), ('svr', SVR())])
print (f'Score {pipe.score(X_test, y_test)}')


Score 0.9102328417218084


In [37]:
#Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

train_pred_rf = rf_model.score(X_train, y_train)

print (f'Score {train_pred_rf}')


Score 0.9522179074654811


In [38]:
# Save the model

from joblib import dump, load
dump(lin_regr_model, 'linear_regression_model.joblib') 

['linear_regression_model.joblib']

In [39]:
# Load the model
loaded_lin_model = load('linear_regression_model.joblib')

In [42]:
score = loaded_lin_model.score(X_test, y_test)
print (f'Score {score}')

Score 0.9189457447498501


In [55]:
# New data to predict
pr = pd.read_csv('data/test_new.csv')


# apply the whole pipeline to data
pred = pipe.predict(X_test)
pd.DataFrame({"Prediction": pred}).reset_index(drop=True).sample(10)

,Prediction
580,0.605353
3055,0.386221
2100,0.523560
1079,0.458632
3354,0.100067
658,0.621940
3964,0.800312
360,0.308516
1597,0.561746
1942,0.406398


In [56]:
#Order of Columns
# Gender: 0 = Male, 1 = Female 
# Company type: 0 = Service, 1 = Product 
# WFH Setup: 0 = No, 1 = Yes 
# Designation = position of emplyee in workplace (range 0-5) high number is high designation
# Resource Allocation = number of working hours (range 1-10) 
# Menatal Fatigue = level of fatigue mentally the employee is facing. (range 0.0-10.0)

new_data = [0, 0, 1, 3, 10, 8.2]

pred = pipe.predict([new_data])
print(f'Burn Rate: {pred}')

Burn Rate: [0.79300679]


In [57]:
new_data = [1, 1, 1, 1, 3, 2.6]

pred = pipe.predict([new_data])
print(f'Burn Rate: {pred}')

Burn Rate: [0.15673477]


In [58]:
new_data = [0, 1, 0, 8, 5, 6.7]

pred = pipe.predict([new_data])
print(f'Burn Rate: {pred}')

Burn Rate: [0.61577179]


In [ ]:
#Ask for user input to calculate Burn Rate.

print('Calculate your Burn Rate. Enter your info.')
print('* Your Burn Rate will be on a scale between 0-1')

user_input = []

while True:
    
    gender = input('Enter your gender. 0 = male, 1 = female')
    user_input.append(gender)
    
    company_type = input('Enter your company type. 0 = service, 1 = product')
    user_input.append(company_type)
    
    wfh = input('Do you have a "Work From Home" setup?. 0 = No, 1 = yes')
    user_input.append(wfh)
    
    designation = input('Rank your position in your workplace. range 0-5. Low - High, whole numbers only')
    user_input.append(designation)
    
    resource_allocation = input('How many hours do you work in a day? range 0-10, whole numbers only')
    user_input.append(resource_allocation)
    
    mental_fatigue = input('Rank your mental fatigue. 0.0 - 10.0')
    user_input.append(mental_fatigue)
    
    break
    
print(f'Your inputs: {user_input}')

pred = pipe.predict([user_input])
print(f'Your Burn Rate is: {pred}')


Calculate your Burn Rate. Enter your info.
* Your Burn Rate will be on a scale between 0-1
